****
# Project1 of FE5222
****
## About this project


- Team **HO Ngok Chao, GAO Jisan, CHENG Toyuan**

- **Summary**: This notebook in **Python 3** is to serve as part of the final report together with source code for your project, stating the logic of the derivation of pricing method (if not discussed in class).

- **Reference**: [Longstaff-Schwartz (2001): "Valuing American Options by Simulation: A Simple Least-Squares Approach." Review of Financial Studies, Vol. 14, 113-147](https://www.google.de/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0CCEQFjAAahUKEwiXtNSZm4rHAhXHOhQKHTjBD3k&url=https%3A%2F%2Fpeople.math.ethz.ch%2F~hjfurrer%2Fteaching%2FLongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf&ei=7PO9VZeOBcf1ULiCv8gH&usg=AFQjCNFQr1r_Cf_pxylg_amU3TFOZVDc8w&sig2=ixZnX_wWQ48G66BMuQTPZA&bvm=bv.99261572,d.d24),
[Least-Square Monte Carlo for American Options in a Python Class](https://github.com/jpcolino/IPython_notebooks/blob/master/Least%20Square%20Monte%20Carlo%20Implementation%20in%20a%20Python%20Class.ipynb)

****
# American Put Options pricing using LSMC 
****

Summarising from the lecture notes while using more generalised timestamp, if the American Put option is alive within the time horizon $[0, T ]$, early exercise is only allowed at discrete times $0 < t_1 < t_2 < ... < t_{M-1} = T$.  For a particular exercise date $t_k$, early exercise is performed if the payoff from immediate exercise exceeds the continuation value.

This continuation value can be expressed as conditional expectation.

- The initial step of the actual algorithm is to determine the cashflow vector $C_{i,{M-1}}$ at the last timestep $t_{M-1}$ whose continuation values are zero. For each stock price path $i$:

\begin{equation}
\begin{array}{lll}
C_{i,t_{M-1}}& =& \max \left(S_{i,t_{M-1}} - K,0\right)\\
\end{array}
\end{equation}


- Second, we need to simulate the **exercise value** at timestamp $t_n$, $𝑛=𝑀−2,…,1$:

\begin{equation} \max \left(K-S_{i,t_n},0\right) > 0 \end{equation} 


- In order to obtain **continuation values**, we regress the discounted future cash-flows realised from continuing (i.e. **Y** in the lecture notes) onto polynomial of the spot price (i.e. **X** in the lecture notes). The regression is done by using the values from all of the stock price paths. The **continuation value** $\widetilde{\mathbb{E}}[V_{i,t_{n+1}}*e^{-rdt} \mid C_{i,t_n}]$ for a stock price path $i$ with values $S_{i,t_n}$ at time $t_{n}$ is

\begin{equation}\widetilde{\mathbb{E}}[C_{V,t_{n+1}}*e^{-rdt} \mid C_{i,t_n}]=\beta_{0}+\beta_{1} C_{i,t_n}+\beta_{2} C_{i,t_n}^{2} + \beta_{3} C_{i,t_n}^{3} + \beta_{4} C_{i,t_J}^{4} + \beta_{5} C_{i,t_n}^{5}\end{equation}



- Once we have the **continuation values** and **exercise values** , we will perform early exercise if exercise value is larger than continuation value and obtain the value of the option at discounted to timestamp $t_n$

\begin{equation}
\begin{array}{lll}
V_{i,t_{n}} = max(C_{i,t_{n-1}}, \widetilde{\mathbb{E}}[V_{i,t_{n+1}}*e^{-rdt} \mid C_{i,t_{n-1}}])\\
\end{array}
\end{equation}


- We repeat this process for backward one step at a time $t_n$, $n= M-2,\ldots,1$. At each time-step, we keep the value of the put option discounted to $t_n$, $V_{i,t_{n}}$


- Finally, we take average of $V_{i,t_1} * e^{-rdt}$, i=1..number of simulation and get the value of the option at time 0

In [51]:
import numpy as np

class AmericanOptionsLSMC(object):
    """ Class for American put options pricing using LSMC
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    simulations: int: need to be even number
    """

    def __init__(self, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.S0 = float(S0)
            self.strike = float(strike)
            assert T > 0
            self.T = float(T)
            assert M > 0
            self.M = int(M)
            assert r >= 0
            self.r = float(r)
            assert div >= 0
            self.div = float(div)
            assert sigma > 0
            self.sigma = float(sigma)
            assert simulations > 0
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Put Options parameters')
            
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(-self.r * self.time_unit)

    @property
    def MCprice_matrix(self, seed = 123):
        """ Returns stock price path matrix with rows: time, columns: stock price path """
        np.random.seed(seed)
        # row 0 for S0, row 1 to M+1 for MC stock price hence in totall M+1 rows
        MCprice_matrix = np.zeros((self.M + 1, self.simulations), dtype=np.float64)
        MCprice_matrix[0,:] = self.S0
        for t in range(1, self.M + 1):
            # Only generate half of what is needed and take both positive and negative of it to obtain the full set
            # note that in the gbm solution here the time difference is self.time_unit has the brownian needs to time that
            brownian = np.random.standard_normal( self.simulations // 2)
            brownian = np.concatenate((brownian, -brownian))
            #GBM solution
            MCprice_matrix[t, :] = (MCprice_matrix[t - 1, :]
                                  * np.exp((self.r - self.sigma ** 2 / 2.) * self.time_unit
                                  + self.sigma * brownian * np.sqrt(self.time_unit)))
        return MCprice_matrix

    @property
    def MCpayoff(self):
        """Returns the exercise value of American Put Option"""
        payoff = np.maximum(self.strike - self.MCprice_matrix,
                        np.zeros((self.M + 1, self.simulations),
                        dtype=np.float64))
        return payoff

    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        # last row's cash flow is already determined to be the exercise value as there is no continuation value
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        # going backward -1 at a time
        # value matrix column t is the cash flow of put option a time t regardless of when it is exercise 
        for t in range(self.M - 1, 0 , -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 5)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)

        return value_matrix[1,:] * self.discount


    @property
    def price(self): return np.sum(self.value_vector) / float(self.simulations)
    
    @property
    def delta(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)
    
    @property
    def gamma(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.delta - myCall_2.delta) / float(2. * diff)
    
    @property
    def vega(self):
        diff = self.sigma * 0.01
        myCall_1 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma + diff, 
                                       self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.S0,
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma - diff, 
                                       self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)    
    
    @property
    def rho(self):        
        diff = self.r * 0.01
        if (self.r - diff) < 0:        
            myCall_1 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r + diff, self.div, self.sigma, 
                                       self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
            return (myCall_1.price - myCall_2.price) / float(diff)
        else:
            myCall_1 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r + diff, self.div, self.sigma, 
                                       self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r - diff, self.div, self.sigma, 
                                       self.simulations)
            return (myCall_1.price - myCall_2.price) / float(2. * diff)
    
    @property
    def theta(self): 
        diff = 1 / 252.
        myCall_1 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T + diff, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.S0, 
                                       self.strike, self.T - diff, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
        return (myCall_2.price - myCall_1.price) / float(2. * diff)
    
        

In [52]:
AmericanPUT = AmericanOptionsLSMC( 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )

In [53]:
AmericanPUT = AmericanOptionsLSMC( 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
print ('Price: ', AmericanPUT.price)
print ('Delta: ', AmericanPUT.delta)
print ('Gamma: ', AmericanPUT.gamma)
print ('Vega:  ', AmericanPUT.vega)
print ('Rho:   ', AmericanPUT.rho)
print ('Theta: ', AmericanPUT.theta)

Price:  4.473117701771221
Delta:  -0.7112251324731934
Gamma:  0.12615233203125087
Vega:   12.196835824506369
Rho:    -10.0335229852333
Theta:  -1.8271728267244622


In [54]:
def prices():
    for S0 in (36., 38., 40., 42., 44.):  # initial stock price values
        for vol in (0.2, 0.4):  # volatility values
            for T in (1.0, 2.0):  # times-to-maturity
                AmericanPUT = AmericanOptionsLSMC(S0, 40., T, 50, 0.06, 0.06, vol, 1500)
                print ("Initial price: %4.1f, Sigma: %4.2f, Expire: %2.1f --> Option Value %8.3f" % (S0, vol, T, AmericanPUT.price))

In [55]:
from time import time
t0 = time()
optionValues = prices()  # calculate all values
t1 = time(); d1 = t1 - t0
print("Duration in Seconds %6.3f" % d1)

Initial price: 36.0, Sigma: 0.20, Expire: 1.0 --> Option Value    4.439
Initial price: 36.0, Sigma: 0.20, Expire: 2.0 --> Option Value    4.779
Initial price: 36.0, Sigma: 0.40, Expire: 1.0 --> Option Value    7.135
Initial price: 36.0, Sigma: 0.40, Expire: 2.0 --> Option Value    8.459
Initial price: 38.0, Sigma: 0.20, Expire: 1.0 --> Option Value    3.225
Initial price: 38.0, Sigma: 0.20, Expire: 2.0 --> Option Value    3.726
Initial price: 38.0, Sigma: 0.40, Expire: 1.0 --> Option Value    6.134
Initial price: 38.0, Sigma: 0.40, Expire: 2.0 --> Option Value    7.666
Initial price: 40.0, Sigma: 0.20, Expire: 1.0 --> Option Value    2.296
Initial price: 40.0, Sigma: 0.20, Expire: 2.0 --> Option Value    2.808
Initial price: 40.0, Sigma: 0.40, Expire: 1.0 --> Option Value    5.201
Initial price: 40.0, Sigma: 0.40, Expire: 2.0 --> Option Value    6.815
Initial price: 42.0, Sigma: 0.20, Expire: 1.0 --> Option Value    1.589
Initial price: 42.0, Sigma: 0.20, Expire: 2.0 --> Option Value  